In [1]:
import pandas as pd
import json
import altair as alt
import numpy as np
from vega_datasets import data as datasets
from scipy.optimize import curve_fit

# Billion Dollar Weather and Climate Disasters

#### Motivation

It comes as no surprise that climate-related disasters have been on the rise throughout the world for the past few decades. In the last few years, in particular, record-shattering disasters seem to have become troublingly commonplace. In 2020, the US witnessed a surprising diversity of such disasters. The climate disasters have been historic not just for their intensity, but also their diversity. In the US, 2020 was marked by California forests engulfed in flames in one of the worst wildfires on record, all the while enduring 30 named storms brewing in the Atlantic. 

#### Billion-Dollar Disasters
In 2020, the US witnessed 22 distinct climate disaster events costing upwards of 1 billion dollars each; causing a cumulative economic damage of around 95 billion dollars (NOAA 2021). While 2020 has been a particularly devasting year, such disasters have been steadly on the rise over the past several years. The National Oceanic and Atmospheric Administration (NOAA, 2021) has been recording such 'billion-dollar disasters' for at least the past four decades. Over the years, these billion-dollar disasters have begun accounting for larger and larger proportions of the total damages from climate related events in the US. In fact, these events account for more than 80% of the damage from all recorded US weather and climate events (NOAA, 2021).

#### The Dataset
In this report, we dig into NOAA's [billion-dollar disaster dataset](https://www.ncdc.noaa.gov/billions/overview) which records the total number of annual billion-dollar disasters from 1980-2020, broken down into seven categories: _droughts, flooding, freeze, severe storms, tropical cyclones, wildfires, and winter storms_. Besines raw counts, the dataset also provides estimates of the total number of deaths and economic cost (in billions of dollars) attributed to each disaster. NOAA also provides this information aggregated over the entire US, as well across various climate regions. The economic cost metrics have been adjusted for inflation with respect to the consumer price index of 2020.

All the data used in this report was obtained from the [Billion-Dollar Weather and Climate Disasters: Time Series](https://www.ncdc.noaa.gov/billions/time-series), followed by some manual data cleaning. Besides that, we rely on Altair for visualization, scipy for some analysis, and this [github gist](https://gist.github.com/rogerallen/1583593) for a useful dictionary to convert between US state names and acronyms. We'll delve into futher details of the dataset as we encounter them

#### References
(NOAA, 2021) NOAA National Centers for Environmental Information (NCEI) U.S. Billion-Dollar Weather and Climate Disasters (2021). https://www.ncdc.noaa.gov/billions/, DOI: 10.25921/stkw-7w73

#### The Questions
We will attempt to answer the following questions, primarily through visualizations:
1. Have billion-dollar climate disasters been occuring more frequently?
2. Have certain types of disasters been occuring more frequently than others?
3. Do certain types of disasters cause more damage (economic and loss of life) than others?
4. Are there any geographic patterns in the changes in frequency and intensity of these disasters? Have certain parts of the US been hit harder than others?


# Load Data

All the data used in this analysis can be found in the `data` folder of [this Github repo](https://github.com/sandeshAdhikary/visualizations/tree/main/A2-extreme-weather). The `Data Wrangling` notebook in the repo was used to clean the original json datasets obtained from NOAA.

Let's start by loading all the datasets we'll be using

In [2]:
# Data on billion-dollar disasters aggregated across the US
usa_data = pd.read_csv('data/usa_data.csv', usecols=['year','count','cost','deaths','disaster'])

# Data on billion-dollar disasters segmented into various climate regions
data = pd.read_csv('data/full_data.csv')

# A dictionary listing what states are included in which climate regions
with open('data/regions.json', 'r') as j:
    regions_dict = json.loads(j.read())
    
# A helpful dictionary to convert between abbreviations and full-forms of US states
with open('data/us_state_abbrv.json', 'r') as j:
    state_abbrv_dict = json.loads(j.read())

## Billion-Dollar Disasters in the US Over the Years

Let's start by looking at the trends in the US in general.

In [3]:
usa_data.head()

,year,count,cost,deaths,disaster
0,1980,1.0,33.9,1260.0,drought
1,1981,0.0,0.0,0.0,drought
2,1982,0.0,0.0,0.0,drought
3,1983,1.0,8.0,0.0,drought
4,1984,0.0,0.0,0.0,drought


Here, the `count` column records the number of times the disaster in the `disaster` column was recorded for the year in the `year` column. The `cost` and `deaths` columns indicate the estimated economic cost (in billions) and number of deaths due to that disaster. This data has been aggregated across all states in the US.

All the columns have the datatypes we'd expect:

In [4]:
print(usa_data.dtypes)

year          int64
count       float64
cost        float64
deaths      float64
disaster     object
dtype: object


There are no null values:

In [5]:
print("There are {} null values in the dataset".format(usa_data.isna().sum().sum()))

There are 0 null values in the dataset


Let's see what disasters are included in this dataset

In [6]:
usa_data['disaster'].unique()

array(['drought', 'flooding', 'freeze', 'severe-storm',
       'tropical-cyclone', 'wildfire', 'winter-storm', 'all-disasters'],
      dtype=object)

Is the `all-disasters` category the sum of all other categories present in the data? Or, are all disaster categories included under `all-disasters` present in our dataset?

In [7]:
alt.layer(
    alt.Chart(usa_data).mark_bar().transform_filter(
        (alt.datum.disaster != 'all-disasters')
    ).encode(
        x = alt.X('year:N',axis=alt.Axis(title='Years', values=np.arange(1980,2021,5), labelAngle=0)),
        y = alt.Y('sum(count):Q',axis=alt.Axis(title='Total Counts Across Disasters [Blue]'))
    ),
    alt.Chart(usa_data).mark_circle(color="#FFAA00",fillOpacity=1.0,size=100).transform_filter(
        (alt.datum.disaster == 'all-disasters')
    ).encode(
        x = 'year:N',
        y = alt.Y('count:Q', axis=alt.Axis(title="Count of 'all-disasters' [Yellow] "))
    )
).properties(
    width=800,
    height=250
).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)

Finally, let's see if we have data on all years for all individual disasters

In [8]:
alt.Chart(usa_data).mark_tick().encode(
    x = alt.X('year:N'),
    y = alt.Y('disaster')
)

alt.Chart(...)

Ok. Looks like we have all individual years and individual disasters covered. Let's move on to some analysis, we'll come back and check the other datasets when we end up using them

### Have billion-dollar climate disasters been occuring more frequently?

In the chart below, we'll plot the number of disasters over the years, and also fit some curves to this data so we can extrapolate to see what we might expect in the next decade or so. We'll fit a few polynomials form $y = ax^n + b$ for $n=[1,2,3,4,5]$ to our data from $1980-2020$, and find the best one based on root mean squared prediction errors (RMSE).

In [9]:
data = usa_data
disaster = 'all-disasters'

# Select out rows from dataset for the disaster
all_disaster_data = data.loc[data['disaster'] == disaster, :].reset_index(drop=True)

# Get metric data as our targets
regression_data = data.loc[data['disaster'] == disaster,'count'].reset_index(drop=True)

# Define a fit function for a given exponent n
def fit_func(n): 
    return lambda x,a,b: a + b*x**n

# Set the list of exponents to try
exponents = {'x': 1, 'x^2': 2,'x^3': 3,'x^4': 4,'x^5': 5}
colors = ['red','blue','green','orange', 'brown']
num_future_yrs = 15

preds = np.zeros((len(exponents),len(regression_data) + num_future_yrs-1))
rmse_errs = np.zeros(len(exponents))

for idx,exp_name in enumerate(exponents.keys()):
    fn = fit_func(exponents[exp_name])
    params, cov = curve_fit(fn, np.arange(1,len(regression_data)+1), regression_data)
    preds[idx,:] = fn(np.arange(1,len(regression_data)+num_future_yrs), params[0], params[1])
    rmse_errs[idx] = np.sqrt(np.sum((preds[idx, 0:len(regression_data)] - regression_data)**2))

# Collect the predictions    
pred_df = pd.DataFrame(preds).T
pred_df.columns = exponents.keys()
pred_df['year'] = np.arange(1980,2020+num_future_yrs)
pred_df = pred_df.reset_index(drop=True).melt('year') # Turn columns to rows

Ok. Let's see which of the polynomials ended up giving us the best fit

In [10]:
rmse_df = pd.DataFrame()
rmse_df['exps'] = list(exponents.keys())
rmse_df['rmse_errs'] = rmse_errs
rmse_plot = alt.Chart(rmse_df).mark_line(point=True).encode(
    x = alt.X('exps',axis=alt.Axis(labelAngle=0, title='Exponents of Polynomial Fits')),
    y = alt.Y('rmse_errs', scale=alt.Scale(zero=False), axis=alt.Axis(title='RMSE Errors'))
).properties(
    width=300,
    height=200
)
rmse_plot

alt.Chart(...)

We see that the cubic fit ($y = a + bx^3$) fits the data best among all the polynomials we've tried. In the chart below, we present the best fit lines for all polynomials, but highlight (via opacity) the cubic fit:

In [11]:

# Plot the raw data for disasters as points
chart = alt.Chart(all_disaster_data).mark_point().encode(
    alt.X('year:N', axis=alt.Axis(title='Years', values=np.arange(1980,2020+num_future_yrs,5), labelAngle=0)),
    alt.Y('{}'.format('count'), axis=alt.Axis(title='Number of Annual Disasters'))
).properties(
    width=850,
    height=450
)

# Plot the fitted curves
fit_charts = alt.Chart(pred_df).mark_line().encode(
    alt.X('year:N'),
    alt.Y('value:Q'),
    color='variable:N',
    opacity = alt.condition(alt.datum.variable == 'x^3', alt.value(1.0), alt.value(0.3))
)

# Add label texts for fit curves
annotations = pd.DataFrame({'x': [2034]*len(exponents), 'y': [73,55,40,29,21], 'text': ['y=x^5', 
                                                                        'y=x^4',
                                                                        'y=x^3',
                                                                        'y=x^2',
                                                                       'y=x']})
fit_label_chart = alt.Chart(annotations).mark_text(size=10).encode(
    x = alt.X('x:N'),
    y = alt.Y('y:Q'),
    text = 'text',
)

# Shade the Observed and Predicted Regions
cutoffs = pd.DataFrame({
    'start': [1979,2021],
    'stop': [2021,2021+num_future_yrs]
})
shade_chart = alt.Chart(
    cutoffs.reset_index()
).mark_rect(
    opacity=0.2
).encode(
    x='start:N',
    x2='stop:N',
    y=alt.value(0),
    y2=alt.value(450),
    color=alt.Color('index:N', legend=None)
)

# Add text for 'observed' and 'predicted'
annotations = pd.DataFrame({'x': [1984,2025], 'y': [70,70], 'text': ['Observed', 'Predicted']})
text_chart = alt.Chart(annotations).mark_text(size=20).encode(
    x = alt.X('x:N'),
    y = alt.Y('y:Q'),
    text = 'text',
)

# Extra comment annotations
annotations = pd.DataFrame({'x': [2020], 'y': [25], 'text': ['Record number of 22 disasters in 2020!']})
comments_chart = alt.Chart(annotations).mark_text(size=12).encode(
    x = alt.X('x:N'),
    y = alt.Y('y:Q'),
    text = 'text',
)


chart + fit_charts + shade_chart + text_chart + fit_label_chart + comments_chart

alt.LayerChart(...)

If the apparent cubic trend in increase in the number of annual disasters were to continue for the next $15$ years, we'd need to brace ourselves for around 35 annual billion-dollar disasters!

### Have certain types of disasters been occuring more frequently than others?

In [12]:
chart = alt.Chart(usa_data).mark_bar().transform_filter(
    alt.datum.disaster != 'all-disasters'
).encode(
    x = alt.X('year:N', axis=alt.Axis(title='Years', values=np.arange(1980,2030,5), labelAngle=0)),
    y = alt.Y('count:Q', axis=alt.Axis(title='Number of Annual Disasters')),
    color = alt.Color('disaster:N'),
    order= alt.Order(
      'count',
      sort='ascending'
    )
)


# Shade the Observed and Predicted Regions
cutoffs = pd.DataFrame({
    'start': [1980,2007],
    'stop': [2007, 2021]
})
shade_chart = alt.Chart(
    cutoffs.reset_index()
).mark_rect(
    opacity=0.2
).encode(
    x='start:N',
    x2='stop:N',
    y=alt.value(0),
    y2=alt.value(50),
    color=alt.Color('index:N', legend=None)
)

# Add comment annotations
annotations = pd.DataFrame({'x': [2010, 2007], 'y': [20, 21], 'text': ['Severe storms begin driving the increase in annual disasters',
                                                                      '2007:']})
comments_chart = alt.Chart(annotations).mark_text(size=12).encode(
    x = alt.X('x:N'),
    y = alt.Y('y:Q'),
    text = 'text',
)

all_charts = (chart + comments_chart + shade_chart).resolve_scale(
    color = 'independent'
)

all_charts


alt.LayerChart(...)

In the last two decades, the bulk of the billion-dollar disasters have been severe storms. While the changes in frequency of severe storms are quite prominent in this plot, it is a bit difficult to see how exactly the frequency of other disasters has been changing over time. We now plot the same data using a different encoding (simple circles) that make these changes in frequency a bit more visceral

In [13]:
bottom_chart = alt.Chart(usa_data).mark_circle().transform_filter(
    alt.datum.disaster != 'all-disasters'
).encode(
    x = alt.X('year:N', axis=alt.Axis(title='Years', values=np.arange(1980,2030,5), labelAngle=0)),
    y = alt.Y('disaster:N',  axis=alt.Axis(title=None)),
    size = 'count'
).properties(
    width = 800,
    height=400
)

top_chart = alt.Chart(usa_data).mark_circle().transform_filter(
    alt.datum.disaster == 'all-disasters'
).encode(
    x = alt.X('year:N', axis=alt.Axis(title=None, values=np.arange(1980,2030,5), labelAngle=0, orient='top')),
    y = alt.Y('disaster:N', axis=alt.Axis(title=None)),
    size = 'count'
).properties(
    width = 800,
    height=30
)


# top_chart | bottom_chart
# + comments_chart
alt.vconcat(top_chart, bottom_chart)

alt.VConcatChart(...)

Some new interesting patterns emerge. Wildfires have been becoming a more regular occurence in the past two decades. Starting 2000, there are fewer spells of years without any wildfires at all.  Freezes and winter-storms seem to have become less frequent in the last two decades. 

The above chart suggests that there have been some marked changes that have occured within the last two decades $2000-2020$. So, let's plot the above annual disaster count data, but using the average from 1980-1999 as a baseline.

In [14]:
cutoff_year = 1999

# Get baseline averages
baseline_data = usa_data.copy().loc[usa_data['year'] < cutoff_year, ['count','disaster']].reset_index(drop=True)
baseline_data = baseline_data.groupby(['disaster']).mean().reset_index()
baseline_data = baseline_data.rename(columns={'count':'avg_count'})

# Add average info and diff from avg to our data
usa_data_avg = usa_data.merge(baseline_data, left_on='disaster',right_on='disaster')
usa_data_avg['avg_diff'] = usa_data_avg['count'] - usa_data_avg['avg_count']

alt.Chart(usa_data_avg).mark_bar().encode(
    x = alt.X('year:N', axis=alt.Axis(title='Years', values=np.arange(1980,2030,5), labelAngle=0)),
    y = alt.Y('avg_diff:Q', axis=alt.Axis(title='Change in Annual Disaster Counts from 1980-1999')),
    color=alt.condition(
        alt.datum.avg_diff > 0,
        alt.value("green"),  # The positive color
        alt.value("red")  # The negative color
)).transform_filter(
    alt.datum.year > cutoff_year
).facet(
    facet = alt.Facet('disaster:N', title=None),
    columns=2
)

alt.FacetChart(...)

We see that most disasters have become more frequent in comparison to the 1980-1999 average, particularly `severe-storms` and `tropical-cyclones`. Two notable exceptions to the rule are ``winter-storms`` and ``freezes``, which have mostly decreased in frequency.

### Do certain types of disasters cause more damage (economic and loss of life) than others?

In [15]:
width = 800

# Death Charts
chart_death = alt.Chart(usa_data).mark_bar().encode(
    x = alt.X('year:N', axis=alt.Axis(title=None, values=np.arange(1980,2030,5), labelAngle=0)),
    y = alt.Y('deaths', axis=alt.Axis(title='Number of Deaths')),
    color='disaster',
    order =  alt.Order(
      'deaths:N',
      sort='ascending'
    )
).transform_filter(
    alt.datum.disaster != 'all-disasters'
).properties(
    width = width
)

# Add comment annotations
# Extra comment annotations
annotations = pd.DataFrame({'x': [1990, 2011], 
                            'y': [1200, 1200], 
                            'text': ['1980-2000: Droughts caused the most fatalities',
                                     '2001-2020: Droughts not as life-threatening']})
comments_chart_death = alt.Chart(annotations).mark_text(size=12).encode(
    x = alt.X('x:N'),
    y = alt.Y('y:Q'),
    text = 'text',
)

# Money Charts
chart_money = alt.Chart(usa_data).mark_bar().encode(
    x = alt.X('year:N', axis=alt.Axis(title=None, values=np.arange(1980,2030,5), labelAngle=0)),
    y = alt.Y('cost', axis=alt.Axis(title='Estimated Economic Cost (in Billion Dollars)')),
    color='disaster',
    order =  alt.Order(
      'cost:N',
      sort='ascending'
    )
).transform_filter(
    alt.datum.disaster != 'all-disasters'
).properties(
    width = width
)


# Timeline Charts
timeline_df = pd.DataFrame()
timeline_df['year'] = usa_data['year'].unique()
events_dict = {1980: 'Historic Heatwaves',
              2005: 'Hurricane Katrina',
              2012: 'Hurricane Sandy',
              2017: 'Hurricane Harvey'}
timeline_df['events'] = timeline_df['year'].map(events_dict)
timeline_df['events_flag'] = 1 - timeline_df['events'].isna()


# Create timeline chart
timeline_chart = alt.Chart(timeline_df).mark_point().encode(
    x = alt.X('year:O', axis=alt.Axis(title=None, values=np.arange(1980,2030,5), labelAngle=0, orient='bottom')),
    size = alt.Size('events_flag', legend=None)
).properties(
    width = width,
    height = 50
)
# Extra comment annotations
annotations = pd.DataFrame({'x': list(events_dict.keys()), 
                            'y': [0.1]*len(events_dict.keys()), 
                            'text': [events_dict[x] for x in events_dict.keys()]})
comments_chart_timeline = alt.Chart(annotations).mark_text(size=12).encode(
    x = alt.X('x:N'),
    y = alt.Y('y:Q', axis=alt.Axis(labels=False, title=None)),
    text = 'text',
)

In [16]:
alt.vconcat(timeline_chart + comments_chart_timeline,
            alt.vconcat(chart_money,
           chart_death + comments_chart_death)).resolve_scale(
    x = 'independent',
    y = 'independent',
)

alt.VConcatChart(...)

In terms of economic costs, we see that there have been numerous tropical cyclones that have been very economically damaging. Remember that these dollar amounts are already adjusted for inflation with respect to the consumer prince index in 2020. 

In terms of total deaths due to these disasters, we see that the first two decades in the dataset ($1980-1999$) was mostly dominated by drougts. But this is no longer the case in the last two decades.

Comparing the two graphs above, we see the both economic costs and deaths have spikes at some notable events. Let's see if there is some correlation between these variables

In [17]:
full_plot = alt.Chart(usa_data).mark_point(size=100).encode(
    alt.X('deaths:Q', axis=alt.Axis(title='Numbers of Deaths')),
    alt.Y('cost:Q', axis=alt.Axis(title='Estimated Economic Cost (in Billion Dollars)')),
    alt.Color('disaster:N')
).transform_filter(
    alt.datum.disaster != 'all-disasters'
).properties(
    width = 240,
    height=500
)

comments_chart_full = alt.Chart(annotations).mark_text(size=12).encode(
    x = alt.X('x:N'),
    y = alt.Y('y:Q'),
    text = 'text',
)

reduced_plot1 = alt.Chart(usa_data).mark_point(size=100).encode(
    alt.X('deaths:Q', axis=alt.Axis(title='Numbers of Deaths')),
    alt.Y('cost:Q', axis=alt.Axis(title=None)),
    alt.Color('disaster:N')
).transform_filter(
    alt.datum.disaster != 'all-disasters'
).transform_filter(
    alt.datum.cost < 200
).properties(
    width = 240,
    height=500
)

reduced_plot2 = alt.Chart(usa_data).mark_point(size=100).encode(
    alt.X('deaths:Q', axis=alt.Axis(title='Numbers of Deaths')),
    alt.Y('cost:Q', axis=alt.Axis(title=None)),
    alt.Color('disaster:N')
).transform_filter(
    alt.datum.disaster != 'all-disasters'
).transform_filter(
    alt.datum.cost < 30
).transform_filter(
    alt.datum.deaths < 400
).properties(
    width = 240,
    height=500
)

In [18]:
alt.hconcat(full_plot, reduced_plot1, reduced_plot2)

alt.HConcatChart(...)

In the left-most plot, we see two devastating outlier tropical cyclones. In the middle plot, we see droughts tend to be the most damaging to human life. In the right-most plot, we don't really see visually discernable correlation between deaths and economic cost.

### Have certain parts of the US been hit harder than others?

In [19]:
# Load state level data
data = pd.read_csv('data/full_data.csv', usecols = ['cost','count','deaths','disaster','region','year'])

# Load regions dictionary
with open('data/regions.json', 'r') as j:
    regions_dict = json.loads(j.read())
    
# Load state abbreviations dictionary
with open('data/us_state_abbrv.json', 'r') as j:
    state_abbrv_dict = json.loads(j.read())
    
# Convert regions_dict into a dataframe, and include state info and full-forms
regions = pd.DataFrame.from_dict(regions_dict).transpose().reset_index()
regions.rename(columns={'index':'region'}, inplace=True)
regions = regions.explode('states')
regions.reset_index(drop=True, inplace=True)
regions['abbrv'] = regions['states'].map(state_abbrv_dict)

Ok. Let's take a peek at what regions are covered in our data

In [20]:
# Get a list of all region names for later
all_region_names = regions['name'].unique()

regions.drop_duplicates('region')[['region','name']].reset_index(drop=True)

,region,name
0,CCR,Central Climate Region
1,ENCCR,East North Central Climate Region
2,NECR,Northeast Climate Region
3,NWCR,Northwest Climate Region
4,SCR,South Climate Region
5,SECR,Southeast Climate Region
6,SWCR,Southwest Climate Region
7,WCR,West Climate Region
8,WNCCR,West North Climate Region
9,GCS,Gulf Coast States


Ok. We have $13$ regions in total. Let's now see what states are contained within these regions by plotting them on a map.

In [21]:
# Pull geographic info on all states from vega datasets
states = alt.topo_feature(datasets.us_10m.url, 'states')
# Pull state IDs from another vega dataset so we can use to do lookups
state_ids = datasets.population_engineers_hurricanes()
# Match ids to state names in our dataset
state_ids = state_ids.loc[:,['state','id']]
state_ids = dict(zip(state_ids.state, state_ids.id))
regions['id'] = regions['states'].map(state_ids)

In [22]:
def chloropleth_map(regions_df, projection='albers'):
    return alt.Chart(states).mark_geoshape(stroke='black').encode(
        alt.Color('name:N', scale=alt.Scale(scheme='tableau20'))
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(regions_df, 'id', list(regions.columns))
    ).properties(
        width=800,
        height=350
    ).project(
        type=projection
    )

chloropleth_map(regions)

alt.Chart(...)

Our dataset only overs the contiguous US; good to know! But there's a problem -- why are there only 11 (and not 13) regions listed in this plot? Maybe this is because some states are counted in multiple regions? Let's check

In [23]:
region_membership = regions.groupby('states')['region'].count().reset_index()
region_membership = region_membership.rename(columns={'region':'region_membership'})
regions_membership = regions.merge(region_membership, on ='states')

heatmap = alt.Chart(regions_membership).mark_rect().encode(
    alt.Y('states:O', axis=alt.Axis(title=None), sort=alt.EncodingSortField(field="region:N", order='ascending')),
    alt.X('region:N'),
    alt.Color('name:N', scale=alt.Scale(scheme='tableau20'), legend=alt.Legend(title='Regions'))
).properties(
    width=400,
    height=700
)

membership_plot = alt.Chart(regions_membership).mark_text().encode(
    y = alt.Y('states:O', axis=alt.Axis(title='Number of regions that includes each state', orient='right')),
    text = alt.Text('region_membership:Q'),
    color = alt.Text('region_membership:Q', legend=None),
).properties(
    height=700
)


heatmap | membership_plot 

alt.HConcatChart(...)

Aha! Just as we suspected, many states are included in multiple regions. After starting at the region names for a bit, we notice that most of these names end with "Climate Region". What if we removed them?

In [24]:
region_names_to_drop = ['Great Lakes States', 'Gulf Coast States', 'Southern Plains', 'Tornado Alley']
reduced_region_names = [x for x in all_region_names if x not in region_names_to_drop]

In [25]:
regions_membership = regions.loc[regions['name'].isin(reduced_region_names),:]
regions_membership = regions_membership.groupby('states')['region'].count().reset_index()
regions_membership = regions_membership.rename(columns={'region':'region_membership'})
regions_membership = regions.loc[regions['name'].isin(reduced_region_names),:].merge(regions_membership, on ='states')


heatmap = alt.Chart(regions_membership).mark_rect().encode(
    alt.Y('states:O', axis=alt.Axis(title=None), sort=alt.EncodingSortField(field="region:N", order='ascending')),
    alt.X('region:N'),
    alt.Color('name:N', scale=alt.Scale(scheme='tableau20'), legend=alt.Legend(title='Regions'))
).properties(
    width=400,
    height=700
)

membership_plot = alt.Chart(regions_membership).mark_text().encode(
    y = alt.Y('states:O', axis=alt.Axis(title='Number of regions that includes each state', orient='right')),
    text = alt.Text('region_membership:Q'),
    color = alt.Text('region_membership:Q', legend=None),
).properties(
    height=700
)


heatmap | membership_plot 

alt.HConcatChart(...)

Great! Now we see that each state is included only in 1 region. There is still one more question: did we end up losing any states when we dropped those extra regions? Let's see

In [26]:
alt.Chart(states).mark_geoshape(stroke='black').encode(
        alt.Color('name:N', scale=alt.Scale(scheme='tableau20'), title='Climate Regions')
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(regions.loc[regions['name'].isin(reduced_region_names)], 'id', list(regions.columns))
    ).properties(
        width=750,
        height=350
    ).project(
        type='albersUsa'
    )

alt.Chart(...)

That seems to have done the trick. We now have all 11 of the climate-regions represented in our map. Also, every state is colored on this cholopleth; we have full coverage.

In [27]:
# Update regions dataset with the reduced region names
regions = regions.loc[regions['name'].isin(reduced_region_names),:]

regions_data = data.merge(regions.loc[:,['region','name']].drop_duplicates(), on='region', how='left')
regions_data = regions_data.loc[regions_data['name'].isin(reduced_region_names),:]

Let's start off by visualizing the total disasters across all the categories over the various regions in our dataset

In [28]:
bottom_chart = alt.Chart(regions_data).mark_circle().encode(
    alt.Y('disaster:N', axis=alt.Axis(title=None)),
    alt.X('region:N'),
    alt.Size('sum(count)')
).properties(
    width=450,
    height=300
).transform_filter(
    alt.datum.disaster != 'all-disasters'
)

top_chart = alt.Chart(regions_data).mark_circle().encode(
    alt.Y('disaster:N', axis=alt.Axis(title=None)),
    alt.X('region:N', axis=alt.Axis(orient='top')),
    alt.Size('sum(count):Q')
).properties(
    width=450,
    height=50
).transform_filter(
    alt.datum.disaster == 'all-disasters'
)


alt.vconcat(top_chart, bottom_chart).resolve_scale(
#     x = 'independent',
    y = 'independent',
)

alt.VConcatChart(...)

In the top plot, we see that some of the hardest hit regions include Cenetral Climate Region (CCR), South Climate Region (SCR), South East Climate Region (SECR). Let's aggregate them as 'High Frequency Disaster Region', as see where they land on the map

In [29]:
df = regions.copy()
df['name'][~df['region'].isin(['CCR','SCR','SECR'])] = 'Other'
df['name'][df['region'].isin(['CCR','SCR','SECR'])] = 'High Frequency Disaster Regions'


alt.Chart(states).mark_geoshape(stroke='black').encode(
        alt.Color('name:N', scale=alt.Scale(scheme='tableau20'))
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(df, 'id', list(regions.columns))
    ).properties(
        width=800,
        height=350
    ).project(
        type='albersUsa'
    )

/Users/sandesh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/sandesh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


alt.Chart(...)

So, we see that the south-west corner of the US has endured a big chunk of the total billion-dollar disasters (from $1980-2020$).

Now, let's take a look at disasters by the years in the different regions

In [30]:
# Plotting the total number of disasters across the regions
alt.Chart(regions_data).mark_circle().transform_filter(
        alt.datum.disaster != 'all-disasters'
    ).encode(
    x = alt.X('year:O',axis=alt.Axis(title='Years', values=np.arange(1980,2030,5), labelAngle=0)),
    size = alt.Size('sum(count):Q', title = 'Annual Disasters'),
    y = alt.Y('name:N',axis=alt.Axis(title=None)),
    color= alt.Color('region:N', title = 'Climate Regions')
).properties(
    width=700,
    height=500
)

alt.Chart(...)

Up until 2000, the western regions of the US did not see billion dollar disasters as frequently; but that has changed in the last two decades.

Finally, let's take a look at how the economic and human-life costs of the various disasters have been distributed across the various climate regions

In [31]:
width = 700

chart_death = alt.Chart(regions_data).mark_bar().transform_filter(
        alt.datum.disaster == 'drought'
    ).encode(
    alt.X('year:O', axis=alt.Axis(title='Years', values=np.arange(1980,2030,5), labelAngle=0)),
    alt.Y('name', title=None),
    alt.Color('sum(deaths)', scale=alt.Scale(scheme='browns'), title='Deaths')
).properties(
    width=width,
    height=450
)

chart_money = alt.Chart(regions_data).mark_bar().transform_filter(
        alt.datum.disaster == 'drought'
    ).encode(
    alt.X('year:O',axis=alt.Axis(title=None, values=np.arange(1980,2030,5), labelAngle=0)),
    alt.Y('name', title=None),
    alt.Color('sum(cost)', scale=alt.Scale(scheme='purples'), title='Cost (Billions)')
).properties(
    width=width,
    height=450
)

# Timeline Charts
timeline_df = pd.DataFrame()
timeline_df['year'] = usa_data['year'].unique()
events_dict = {1980: 'Historic Heatwaves',
              2005: 'Hurricane Katrina',
              2012: 'Hurricane Sandy',
              2017: 'Hurricane Harvey'}
timeline_df['events'] = timeline_df['year'].map(events_dict)
timeline_df['events_flag'] = 1 - timeline_df['events'].isna()


# Create timeline chart
timeline_chart = alt.Chart(timeline_df).mark_point().encode(
    x = alt.X('year:O', axis=alt.Axis(title=None, values=np.arange(1980,2030,5), labelAngle=0)),
    size = alt.Size('events_flag', legend=None)
).properties(
    width = width,
    height = 50
)
# Extra comment annotations
annotations = pd.DataFrame({'x': list(events_dict.keys()), 
                            'y': [0.1, 0.1, 0.1, 0.05], 
                            'text': [events_dict[x] for x in events_dict.keys()]})
comments_chart_timeline = alt.Chart(annotations).mark_text(size=12).encode(
    x = alt.X('x:N', axis=alt.Axis(title=None)),
    y = alt.Y('y:Q', axis=alt.Axis(labels=False, title=None)),
    text = 'text',
).properties(
    width = width
)

In [32]:
alt.vconcat(timeline_chart + comments_chart_timeline, chart_money, chart_death).resolve_scale(
    color='independent'
)

alt.VConcatChart(...)

Compared to our earlier charts with data aggregated across the US, we now see how some of the big disasters affected different regions differently. In the aggregated US data, we saw big spikes in economic cost at 2005 (probably Hurricane Katrina). Here, we see that this damage was largely concentrated in a single region, the South Climate Region (where Katrina hit the hardest). In general, the above plot shows us how certain disaster years have been devastating across the board (e.g. the heatwaves/droughts of 1980), while others are concentrated in specific regions.